In [2]:
import os
from google import genai
from google.genai import types
import time
from dotenv import load_dotenv

load_dotenv()
client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))

In [16]:
# Create the File Search store with an optional display name
file_search_store = client.file_search_stores.create(config={'display_name': 'TestFileSearchStore'})
print(file_search_store)

name='fileSearchStores/testfilesearchstore-2ilrlghgrj35' display_name='TestFileSearchStore' create_time=datetime.datetime(2025, 11, 19, 21, 7, 53, 580710, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 19, 21, 7, 53, 580710, tzinfo=TzInfo(0)) active_documents_count=None pending_documents_count=None failed_documents_count=None size_bytes=None


In [9]:
# Get File Search store
my_file_search_store_name = "fileSearchStores/testfilesearchstore-4nfrvuod0yei"
file_search_store = client.file_search_stores.get(name=my_file_search_store_name)
print(file_search_store)

name='fileSearchStores/testfilesearchstore-4nfrvuod0yei' display_name='TestFileSearchStore' create_time=datetime.datetime(2025, 11, 19, 20, 32, 3, 467919, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 19, 20, 32, 3, 467919, tzinfo=TzInfo(0)) active_documents_count=2 pending_documents_count=None failed_documents_count=None size_bytes=180752


In [5]:
# Convert files from .pdf to .md
import pymupdf4llm

md_files = []

for file in os.listdir('./files/original'):
    md_files.append({
        'content': pymupdf4llm.to_markdown(f'./files/original/{file}'),
        'display_name': file.split('.')[0]
    })

print(md_files[0]['display_name'])
print(md_files[0]['content'][:500])

test-doc-2
# **Opinion 28/2024 on certain data protection aspects related** **to the processing of personal data in the context of AI** **models**

## **Adopted on 17 December 2024**

Adopted 1


**Executive summary**


AI technologies create many opportunities and benefits across a wide range of sectors and social

activities.


By protecting the fundamental right to data protection, GDPR supports these opportunities and

promotes other EU fundamental rights, including the right to freedom of thought, exp


In [6]:
# Store the md files in files/processed
for file in md_files:
    with open(f'./files/processed/{file["display_name"]}.md', 'w') as f:
        f.write(file['content'])


In [17]:
# Upload and import a file into the File Search store, supply a file name which will be visible in citations
for file in os.listdir('./files/processed'):
    operation = client.file_search_stores.upload_to_file_search_store(
    file=f"./files/processed/{file}",
    file_search_store_name=file_search_store.name,
    config={
        'display_name' : file.split('.')[0],
    }
)

In [18]:
# List all file search stores
for file_search_store in client.file_search_stores.list():
    print(file_search_store)


name='fileSearchStores/testfilesearchstore-2ilrlghgrj35' display_name='TestFileSearchStore' create_time=datetime.datetime(2025, 11, 19, 21, 7, 53, 580710, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 19, 21, 7, 53, 580710, tzinfo=TzInfo(0)) active_documents_count=2 pending_documents_count=None failed_documents_count=None size_bytes=180752


In [19]:
# List files in a specific file search store
files_in_store = client.file_search_stores.documents.list(parent=file_search_store.name)
for file in files_in_store:
    print(file)


name='fileSearchStores/testfilesearchstore-2ilrlghgrj35/documents/testdoc1-t6t25xxaxc73' display_name='test-doc-1' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=53776 mime_type='text/markdown' create_time=datetime.datetime(2025, 11, 19, 21, 7, 59, 303432, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2025, 11, 19, 21, 8, 0, 266039, tzinfo=TzInfo(0))
name='fileSearchStores/testfilesearchstore-2ilrlghgrj35/documents/testdoc2-337k2a12baah' display_name='test-doc-2' state=<DocumentState.STATE_ACTIVE: 'STATE_ACTIVE'> size_bytes=126976 mime_type='text/markdown' create_time=datetime.datetime(2025, 11, 19, 21, 8, 1, 268491, tzinfo=TzInfo(0)) custom_metadata=None update_time=datetime.datetime(2025, 11, 19, 21, 8, 2, 768519, tzinfo=TzInfo(0))


In [15]:
# Delete all file search stores
for file_search_store in client.file_search_stores.list():
    client.file_search_stores.delete(name=file_search_store.name, config={'force': True})

In [20]:
# LLM Prompt and Output Schema
contents = """
**1. Policy Analyst Role:** You are a specialized policy analyst focused on German health data governance and the framework of **Health Data Justice (HDJ)**, which includes the sub-themes of Fairness, Transparency, Accountability, and Inclusion/Equity.

**2. File Search Query:** Please **search the provided documents** in the File Search tool for any regulations, statutes, or principles related to **health data use, consent, access, or governance** that specifically touch upon the themes of **Fairness, Transparency, Accountability, or Inclusion/Equity** within the German context.

**3. Output Task:** For every distinct and relevant section found, extract the required information and return it strictly according to the provided JSON Schema.
    * **CRITICAL:** For the `source_excerpt_text` field, you **must** use the verbatim text chunk retrieved by the File Search tool that supports your finding.
    * **CRITICAL:** For the `source_uri` field, use the full URI of the document that provided the excerpt.
    * Don't rely on your own knowledge, only use the provided documents.
    * If no relevant sections are found, return an empty array for `analysis_results`.
"""


json_schema = {
  "type": "object",
  "properties": {
    "analysis_results": {
      "type": "array",
      "description": "A list of structured findings from the policy document analysis.",
      "items": {
        "type": "object",
        "properties": {
          "relevance_score": {
            "type": "integer",
            "description": "The relevance score (1-5) of this section to Health Data Justice. (5=Direct Match, 4=Strong Implied Match, 3=Peripheral Match)."
          },
          "hdj_themes": {
            "type": "array",
            "description": "One or more Health Data Justice themes addressed: Fairness, Transparency, Accountability, Inclusion/Equity.",
            "items": {
              "type": "string"
            }
          },
          "excerpt_summary": {
            "type": "string",
            "description": "A concise one-sentence summary of the policy's action or principle found in the retrieved excerpt."
          },
          "source_excerpt_text": {
            "type": "string",
            "description": "The exact block of text (the excerpt/chunk) retrieved from the document that supports this finding."
          },
          "source_uri": {
            "type": "string",
            "description": "The full URI of the document where the excerpt was found."
          }
        },
        "required": [
            "relevance_score",
            "hdj_themes",
            "excerpt_summary",
            "source_excerpt_text",
            "source_uri"
        ]
      }
    }
  },
  "required": ["analysis_results"]
}

In [21]:
# Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=types.GenerateContentConfig(
        response_schema=json_schema,
        response_mime_type="application/json",
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ],
    )
)

print(response.text)

Here is the JSON requested:
```json
{
  "analysis_results": [
    {
      "relevance_score": 5,
      "hdj_themes": [
        "Accountability",
        "Fairness",
        "Transparency",
        "Inclusion/Equity"
      ],
      "excerpt_summary": "The European Health Data Space (EHDS) aims to empower individuals with control over their health data and foster a single market for health data, emphasizing data quality, security, and interoperability while ensuring trust and ethical use.",
      "source_excerpt_text": "The European Health Data Space (EHDS) is a health-specific ecosystem comprising rules, common standards and practices, infrastructures and a framework of governance. It aims to promote persons’ health, provide them with high-quality health care and empower them by giving them more control over their health data. It also seeks to unleash the single market for digital health services and products, as well as to create a real single market for health data. The EHDS is one of 

In [22]:
print(response.candidates[0].grounding_metadata)

None
